In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
print("done")

done


In [18]:
train_texts = pd.read_csv("datasets/salary-train.csv")
test_texts = pd.read_csv("datasets/salary-test-mini.csv")
train_texts.iloc[:, 0] = train_texts.iloc[:, 0].apply(str.lower)
test_texts.iloc[:, 0] = test_texts.iloc[:, 0].apply(str.lower)
train_texts['FullDescription'] = train_texts['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
test_texts['FullDescription'] = test_texts['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [19]:
train_texts.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [20]:
vectorised = TfidfVectorizer(min_df=5)
desc_train = vectorised.fit_transform(train_texts["FullDescription"])
desc_test = vectorised.transform(test_texts["FullDescription"])
desc_train

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [21]:
train_texts["ContractTime"].fillna('nan', inplace=True)
train_texts["LocationNormalized"].fillna('nan', inplace=True)
test_texts[["LocationNormalized", "ContractTime"]].fillna('nan', inplace=True)

c:\users\admin\.virtualenvs\pandas_trying-m-zk6hwn\lib\site-packages\pandas\core\frame.py:4322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [26]:
dict_vect = DictVectorizer()
train_categ = dict_vect.fit_transform(train_texts[['LocationNormalized', 'ContractTime']].to_dict('records'))
test_categ = dict_vect.transform(test_texts[['LocationNormalized', 'ContractTime']].to_dict('records'))
train_categ

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [27]:
X = hstack([desc_train, train_categ])
X_test = hstack([desc_test, test_categ])

In [28]:
ridge = Ridge(alpha=1, random_state=241)
ridge.fit(X, train_texts["SalaryNormalized"])


Ridge(alpha=1, random_state=241)

In [29]:
y_test = ridge.predict(X_test)
y_test

array([56576.52286114, 37136.22493508])

In [30]:
for i in y_test:
    print(round(i, 2))



56576.52
37136.22
